In [3]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

# Toy example 1 - Single eligible group, Single ineligible group

In [ ]:
def _extract_solution_discount(m):
    """
    Get solution from optimization model
    """
    users_in = [v.x for v in m.getVars() if v.VarName.find("ineligible_val") != -1]
    users_el = [v.x for v in m.getVars() if v.VarName.find("eligible_allocation") != -1]
    edge_flows = [v.x for v in m.getVars() if v.VarName.find("edge_flows") != -1]
    
    users_in = np.reshape(users_in, (2, T))
    users_el = np.reshape(users_el, (2, T))
    edge_flows = np.reshape(edge_flows, (2, T))

    solution = {
        "users_in": users_in,
        "users_el": users_el,
        "edge_flows": edge_flows
    }
    
    return solution

In [72]:
def avg_travel_times_1(solution_set, theta):
    
    users_in = solution_set["users_in"]
    users_el = solution_set["users_el"]
    edge_flows = solution_set["edge_flows"]
    
    avg_travel_time_in = sum( users_in[j, t] * (theta[j,0] * edge_flows[j, t] + theta[j,1]) for j in range(2) for t in range(T)) / T
    avg_travel_time_el = sum( users_el[j, t] * (theta[j,0] * edge_flows[j, t] + theta[j,1]) for j in range(2) for t in range(T)) / T
    avg_travel_time_all = sum( edge_flows[j, t] * (theta[j,0] * edge_flows[j, t] + theta[j,1]) for j in range(2) for t in range(T)) / (T*2)
    
    avg_travel_times = {
        "users_in": avg_travel_time_in,
        "users_el": avg_travel_time_el,
        "all": avg_travel_time_all
    }
    
    return avg_travel_times


In [68]:
def m_d1_solve(theta, T, toll, vot_ineligible, vot_eligible, discount_ratio):
    a_1 = theta[0, 0]
    b_1 = theta[0, 1]
    a_2 = theta[1, 0]
    b_2 = theta[1, 1]
    
    # Initialize model
    m_d1 = gp.Model()

    users_in = m_d1.addVars(2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
    edge_flows = m_d1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)
    users_el = m_d1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)

    ## Add constraints to Model 

    # Total edge flow, express and general purpose lanes:
    m_d1.addConstrs((edge_flows[j, t] == users_in[j, t] + users_el[j, t] for j in range(2) for t in range(T)))

    # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
    m_d1.addConstrs((users_in[j, t]>=0 for j in range(2) for t in range(T)))
    m_d1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))

    # Every eligible and ineligible user is assigned to one of three options: \
    # (express paying, express with credit, non-express)
    m_d1.addConstrs((sum(users_in[j, t] for j in range(2)) == 1 for t in range(T)))
    m_d1.addConstrs((sum(users_el[j, t] for j in range(2)) == 1 for t in range(T)))

    ## To edit below:
    # Set Objective
    m_d1.setObjective(sum( a_1*(edge_flows[0, t]) + b_1*(edge_flows[0, t]**2)/2  
                        + a_2*(edge_flows[1, t]) + b_2*(edge_flows[1, t]**2)/2 
                        + toll*users_in[0,t]/vot_ineligible
                        + (1 - discount_ratio)*toll*users_el[0,t]/vot_eligible 
        for t in range(T)), GRB.MINIMIZE)
    
    m_d1.update()

    m_d1.optimize()
    
    solution_set_d1 = _extract_solution_discount(m_d1)

    return solution_set_d1

In [69]:
def m_b1_solve(theta, T, toll, vot_ineligible, vot_eligible, budget):
    a_1 = theta[0, 0]
    b_1 = theta[0, 1]
    a_2 = theta[1, 0]
    b_2 = theta[1, 1]
    
    # Initialize model
    m_b1 = gp.Model()

    users_in = m_b1.addVars(2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
    edge_flows = m_b1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)
    users_el = m_b1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)

    ## Add constraints to Model 

    # Total edge flow, express and general purpose lanes:
    m_b1.addConstrs((edge_flows[j, t] == users_in[j, t] + users_el[j, t] for j in range(2) for t in range(T)))

    # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
    m_b1.addConstrs((users_in[j, t]>=0 for j in range(2) for t in range(T)))
    m_b1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))
    
    # Budget constraint satisfaction:
    if toll != 0:
        m_b1.addConstr((sum(toll*users_el[0, t] for t in range(T)) <= budget))

    # Every eligible and ineligible user is assigned to one of three options: \
    # (express paying, express with credit, non-express)
    m_b1.addConstrs((sum(users_in[j, t] for j in range(2)) == 1 for t in range(T)))
    m_b1.addConstrs((sum(users_el[j, t] for j in range(2)) == 1 for t in range(T)))

    ## To edit below:
    # Set Objective
    m_b1.setObjective(sum( a_1*(edge_flows[0, t]) + b_1*(edge_flows[0, t]**2)/2  
                        + a_2*(edge_flows[1, t]) + b_2*(edge_flows[1, t]**2)/2 
                        + toll*users_in[0,t]/vot_ineligible
        for t in range(T)), GRB.MINIMIZE)
    
    m_b1.update()

    m_b1.optimize()
    
    solution_set_b1 = _extract_solution_discount(m_b1)

    return solution_set_b1

In [70]:
# Affine lane functions:
a_1 = 1.0
b_1 = 1.0
a_2 = 1.0
b_2 = 1.0
theta = np.array([[a_1, b_1], [a_2, b_2]])
T = 1
toll = 1.0
budget = 0.25
vot_ineligible = 1
vot_eligible = 1
discount_ratio = budget / (toll * T)

# Return solution:

solution_set_d1 = m_d1_solve(theta, T, toll, vot_ineligible, vot_eligible, discount_ratio)

solution_set_b1 = m_b1_solve(theta, T, toll, vot_ineligible, vot_eligible, budget)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 6 columns and 14 nonzeros
Model fingerprint: 0x9b774ec7
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 1e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.04770777e+07 -7.5

In [79]:
edge_flows_d1 = solution_set_d1["edge_flows"]
users_el_d1 = solution_set_d1["users_el"]
users_in_d1 = solution_set_d1["users_in"]

edge_flows_b1 = solution_set_b1["edge_flows"]
users_el_b1 = solution_set_b1["users_el"]
users_in_b1 = solution_set_b1["users_in"]

print("edge_flows_d1:\n", edge_flows_d1)
print()
print("users_el_d1:\n", users_el_d1)
print()
print("users_in_d1:\n", users_in_d1)
print()

print()
print("edge_flows_b1:\n", edge_flows_b1)
print()
print("users_el_b1:\n", users_el_b1)
print()
print("users_in_b1:\n", users_in_b1)
print()

avg_travel_times_d1 = avg_travel_times_1(solution_set_d1, theta)
avg_travel_times_b1 = avg_travel_times_1(solution_set_b1, theta)

print()
print("avg_travel_times_d1:\n", avg_travel_times_d1)
print()
print("avg_travel_times_b1:\n", avg_travel_times_b1)
print()

edge_flows_d1:
 [[0.625]
 [1.375]]

users_el_d1:
 [[0.625]
 [0.375]]

users_in_d1:
 [[8.62321325e-11]
 [1.00000000e+00]]


edge_flows_b1:
 [[0.50000006]
 [1.49999994]]

users_el_b1:
 [[0.25]
 [0.75]]

users_in_b1:
 [[0.25000006]
 [0.74999994]]


avg_travel_times_d1:
 {'users_in': 2.37500000005356, 'users_el': 1.906250000123682, 'all': 2.1406250000886757}

avg_travel_times_b1:
 {'users_in': 2.249999907276729, 'users_el': 2.2499999693266233, 'all': 2.2499999383017024}



avg_travel_times_d1:
 {'users_in': 2.37500000005356, 'users_el': 1.906250000123682, 'all': 2.1406250000886757}

avg_travel_times_b1:
 {'users_in': 2.249999907276729, 'users_el': 2.2499999693266233, 'all': 2.2499999383017024}


# From other files:

In [4]:
# # Solver 2, Discount:

# def OptPL_discount(toll, discount_ratio, T = T, num_eligible = num_eligible, num_ineligible = num_ineligible, 
#             vot_ineligible = vot_ineligible, alpha = bpr_true_alpha, fftt_1 = bpr_true_fftt, 
#               fftt_2 = bpr_true_fftt, c_1 = bpr_true_capacity, c_2 = bpr_true_capacity, 
#              a = const_multiplier, cap_thresh = cap_thresh_multiplier, b_1 = apx_slope, b_2 = apx_slope):
#     """
#     Function to solve convex optimization problem given a particular toll and budget value
#     """
    
#     # Initialize model
#     m1 = gp.Model()
#     # Add variables to model
# #     users_in = m1.addVars(num_ineligible, 3, T, name = 'ineligible_val', lb = 0.0, ub = 2.0)
# #     edge_flows = m1.addVars(2, T, name = 'edge_flows', lb = 0.0, ub = num_ineligible * 2.0)
# #     users_el = m1.addVars(3, T, name = 'eligible_allocation', lb = 0.0, ub = 2.0)
#     # Add variables for piecewise affine approximation
# #     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = 0.0, ub = num_ineligible * 2.0)
    
# #     # Add variables for piecewise affine approximation
# #     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = 0.0, ub = num_ineligible * 2.0)
#     users_in = m1.addVars(num_ineligible, 2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
#     edge_flows = m1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)
#     users_el = m1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)
#     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = -10000.0, ub = 10000.0)   

#     ## Add constraints to Model 
    
#     # Total edge flow, express and general purpose lanes:
#     m1.addConstrs((edge_flows[j, t] == sum(users_in[i, j, t] for i in range(num_ineligible)) + \
#                   users_el[j, t] for j in range(2) for t in range(T)))
    
#     # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
#     m1.addConstrs((users_in[i, j, t]>=0 for i in range(num_ineligible) for j in range(2) for t in range(T)))
#     m1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))
    
#     # Every eligible and ineligible user is assigned to one of three options: \
#     # (express paying, express with credit, non-express)
#     m1.addConstrs((sum(users_in[i, j, t] for j in range(2)) == 1 for i in range(num_ineligible) for t in range(T)))
#     m1.addConstrs((sum(users_el[j, t] for j in range(2)) == num_eligible for t in range(T)))
        
#     # Piecewise affine approximation:
#     m1.addConstrs((eps_flows[j, t] >= 0 for j in range(2) for t in range(T))) # Must be at least 0
#     m1.addConstrs((eps_flows[0, t] >= edge_flows[0, t] - cap_thresh*c_1 for t in range(T))) # Must be at least flow - capacity
#     m1.addConstrs((eps_flows[1, t] >= edge_flows[1, t] - cap_thresh*c_2 for t in range(T))) # Must be at least flow - capacity
    
    
#     ## To edit below:
#     # Set Objective
#     m1.setObjective(sum( a*fftt_1*(edge_flows[0, t]) + b_1*(eps_flows[0, t]**2)/2  
#                         + a*fftt_2*(edge_flows[1, t]) + b_2*(eps_flows[1, t]**2)/2 
#                         + sum( toll*users_in[i,0,t]/vot_ineligible[i,t] for i in range(num_ineligible))
#                         + sum( (1 - discount_ratio)*toll*users_el[0,t]/vot_eligible[i,t] for i in range(num_eligible))
#         for t in range(T)), GRB.MINIMIZE)
    
#     m1.update()
    
#     return m1

# Scratch Work:

In [ ]:
# m_sample = gp.Model()
